### This notebook is used to train NN models and write result

In [7]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (12, 6)
import tensorflow as tf
from sklearn import preprocessing
from keras.layers import Dense, Input, BatchNormalization
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
from keras import regularizers
from keras.utils import plot_model
from scipy.stats import linregress
probit = np.array([-1.28155, -0.84162, -0.52440, -0.25335])
xx = np.concatenate((probit, np.array([0]), -np.flip(probit, axis = -1)))

In [8]:
def model(input_shape):
    X_input = Input(shape = input_shape)
    X = Dense(32, activation='relu')(X_input)
    X = BatchNormalization()(X)
    X = Dense(32, activation='relu')(X)
    X = BatchNormalization()(X)
    X = Dense(32, activation='relu')(X)
    X = BatchNormalization()(X)

    X = Dense(9)(X)
    model = Model(inputs = X_input, outputs = X)
    
    return model

In [9]:
def pinball(y_true, y_pred):
    tao = K.cast(np.reshape((np.array(range(9)) + 1)/10, (-1,1)), 'float32')
    pin = K.dot(K.maximum(y_true - y_pred, 0), tao) + K.dot(K.maximum(y_pred - y_true, 0), 1. - tao)
    return K.mean(pin)

#### region = region name, j  = index of the current model (j = 1, 2, ..., 5 in the paper)

In [10]:
# ME NH VT RI SEMASS WCMASS NEMASSBOST
region = 'NEMASSBOST'
j=1

### Train and write result

In [11]:
df_train = pd.read_csv('Data/'+region+'_train.csv')
df_train['HourOfYear'] = pd.DatetimeIndex(df_train['Date']).dayofyear * 24 + df_train['Hour'] - 24
df_dev = pd.read_csv('Data/'+region+'_dev.csv')
df_dev2 = pd.read_csv('Data/'+region+'_dev2.csv')
df_dev['HourOfYear'] = pd.DatetimeIndex(df_dev['Date']).dayofyear * 24 + df_dev['Hour'] - 24
df_dev2['HourOfYear'] = pd.DatetimeIndex(df_dev2['Date']).dayofyear * 24 + df_dev2['Hour'] - 24

load_scaler = preprocessing.StandardScaler().fit(df_train['DEMAND'].values.reshape(-1,1))
for cols in ['DEMAND', '1D', '2D', '3D', '4D', '5D', '6D', '7D', 'D2', 'D3', 'D4', 'D5']:
# for cols in ['DEMAND', 'D1', 'D5', 'DD1', 'DD5', 'W1', 'W5']:
    df_train[cols] = load_scaler.transform(df_train.loc[:,cols].values.reshape(-1, 1))
    df_dev[cols] = load_scaler.transform(df_dev.loc[:,cols].values.reshape(-1, 1))
    df_dev2[cols] = load_scaler.transform(df_dev2.loc[:,cols].values.reshape(-1, 1))

for cols in ['Weekday', 'Weekend?', 'Hour', 'Month']:
#for cols in ['Weekday', 'Weekend?', 'HourOfYear', 'Day']:
    cataScaler = preprocessing.StandardScaler()
    df_train[cols] = cataScaler.fit_transform(df_train.loc[:,cols].values.reshape(-1, 1))
    df_dev[cols] = cataScaler.transform(df_dev.loc[:,cols].values.reshape(-1, 1))
    df_dev2[cols] = cataScaler.transform(df_dev2.loc[:,cols].values.reshape(-1, 1))
    
df_nn_train = df_train[['DEMAND', '1D', '2D', '3D', '4D', '5D', '6D', '7D','D2', 'D3', 'D4', 'D5', 'Weekday', 'Weekend?', 'Hour', 'Month']]
df_nn_dev = df_dev[['DEMAND', '1D', '2D', '3D', '4D', '5D', '6D', '7D','D2', 'D3', 'D4', 'D5', 'Weekday', 'Weekend?', 'Hour', 'Month']]
df_nn_dev2 = df_dev2[['DEMAND', '1D', '2D', '3D', '4D', '5D', '6D', '7D','D2', 'D3', 'D4', 'D5', 'Weekday', 'Weekend?', 'Hour', 'Month']]

Y = df_nn_train['DEMAND'].values.reshape([-1,1])
X = df_nn_train.drop(['DEMAND'], axis = 1).values

Y_s = df_nn_dev['DEMAND'].values.reshape([-1,1])
X_s = df_nn_dev.drop(['DEMAND'], axis = 1).values

Y_r = df_nn_dev2['DEMAND'].values.reshape([-1,1])
X_r = df_nn_dev2.drop(['DEMAND'], axis = 1).values

model = model(input_shape = (15, ))
opt = Adam(lr = 0.0003)
model.compile(loss=pinball, optimizer=opt, metrics=[])

model.fit(X, Y, batch_size = 1024, epochs=220, validation_data=(X_s, Y_s))

Y_r_pred = model.predict(X_r)
Y_r_pred = load_scaler.inverse_transform(Y_r_pred)

result = np.zeros((Y_r_pred.shape[0],2))
for i in range(Y_r_pred.shape[0]):
    slope, intercept, r_value, p_value, std_err = linregress(xx, Y_r_pred[i])
    result[i,0] = intercept
    result[i,1] = slope

df_result = pd.DataFrame(result, columns = ['mean', 'std'])
df_result.to_csv('results/NN_'+region+'_result'+str(j)+'.csv', index = False)

/Users/Tianyi/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Train on 8731 samples, validate on 840 samples
Epoch 1/220
8731/8731 [==============================] - 1s 154us/step - loss: 5.2490 - val_loss: 5.0405
Epoch 2/220
8731/8731 [==============================] - 0s 5us/step - loss: 4.8041 - val_loss: 4.7338
Epoch 3/220
8731/8731 [==============================] - 0s 5us/step - loss: 4.4162 - val_loss: 4.6113
Epoch 4/220
8731/8731 [==============================] - 0s 5us/step - loss: 4.0692 - val_loss: 4.5843
Epoch 5/220
8731/8731 [==============================] - 0s 5us/step - loss: 3.7444 - val_loss: 4.6212
Epoch 6/220
8731/8731 [==============================] - 0s 5us/step - loss: 3.4469 - val_loss: 4.8164
Epoch 7/220
8731/8731 [==============================] - 0s 5us/step - loss: 3.1708 - val_loss: 5.0376
Epoch 8/220
8731/8731 [==============================] - 0s 5us/step - loss: 2.9396 - val_loss: 5.1490
Epoch 9/220
8731/8731 [==============================] - 0s 5us/step - loss: 2.7423 - val_loss: 5.2284
Epoch 10/220
8731/8731 [

8731/8731 [==============================] - 0s 5us/step - loss: 0.9766 - val_loss: 1.1934
Epoch 158/220
8731/8731 [==============================] - 0s 5us/step - loss: 0.9757 - val_loss: 1.1987
Epoch 159/220
8731/8731 [==============================] - 0s 5us/step - loss: 0.9779 - val_loss: 1.2037
Epoch 160/220
8731/8731 [==============================] - 0s 5us/step - loss: 0.9709 - val_loss: 1.1958
Epoch 161/220
8731/8731 [==============================] - 0s 5us/step - loss: 0.9667 - val_loss: 1.1896
Epoch 162/220
8731/8731 [==============================] - 0s 5us/step - loss: 0.9716 - val_loss: 1.2149
Epoch 163/220
8731/8731 [==============================] - 0s 5us/step - loss: 0.9663 - val_loss: 1.1850
Epoch 164/220
8731/8731 [==============================] - 0s 5us/step - loss: 0.9631 - val_loss: 1.1946
Epoch 165/220
8731/8731 [==============================] - 0s 5us/step - loss: 0.9666 - val_loss: 1.2174
Epoch 166/220
8731/8731 [==============================] - 0s 5us/ste